## Deployment of ML models in Heroku using FLASK -2

In [1]:
import pandas as np

In [14]:
df=pd.read_csv('spam.csv',encoding="latin-1")

<IPython.core.display.Javascript object>

In [4]:
df

,class,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [15]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True,axis=1)

In [16]:
df

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   class    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [12]:
df['class']=pd.get_dummies(df['class'],drop_first=True).astype(int)

<IPython.core.display.Javascript object>

In [13]:
df

,class,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [17]:
df['class']=df['class'].map({'ham':0,'spam':1})

In [18]:
df

,class,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [29]:
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [30]:
len(corpus)

5572

In [36]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

In [37]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [38]:
len(words)

5564

In [43]:
corpus_filtered = []
y_filtered = []

for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    
    if review:  # Only keep non-empty reviews
        corpus_filtered.append(review)
        y_filtered.append(df['class'][i])

In [46]:
len(corpus_filtered),len(y_filtered)

(5564, 5564)

In [47]:
type(corpus_filtered),type(y_filtered)

(list, list)

In [49]:
y= pd.Series(y_filtered)

<IPython.core.display.Javascript object>

In [51]:
y.head()

0    0
1    0
2    1
3    0
4    0
dtype: int64

In [120]:
## Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words)

TypeError: object of type 'Word2Vec' has no len()

In [53]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [54]:
from tqdm import tqdm

In [55]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|                                                                                                      | 0/5564 [00:00<?, ?it/s]C:\Users\10734646\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\10734646\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|█████████████████████████████████████████████████████████████████████████████████████████| 5564/5564 [00:00<00:00, 7324.25it/s]


In [56]:
len(X),type(X)

(5564, list)

In [57]:
##independent Features
X_new=np.array(X,dtype=object)

In [59]:
pd.Series(X)

<IPython.core.display.Javascript object>

0       [-0.09669387, 0.24740793, 0.007466938, 0.07024...
1       [-0.08140606, 0.20915096, 0.0037713302, 0.0694...
2       [-0.103763476, 0.2711387, 0.0014583218, 0.0808...
3       [-0.14116888, 0.35347313, 0.009103978, 0.11079...
4       [-0.10997627, 0.27837792, 0.011766829, 0.07873...
                              ...                        
5559    [-0.12169434, 0.31341186, 0.010547594, 0.08595...
5560    [-0.116994, 0.28587425, 0.01857164, 0.07827341...
5561    [-0.051549833, 0.12120441, -0.0041314843, 0.03...
5562    [-0.10869463, 0.281246, 0.0025494702, 0.078581...
5563    [-0.11765301, 0.27791953, 0.0070242, 0.0837444...
Length: 5564, dtype: object

In [60]:
## this is the final independent features
df_list = []

for i in range(len(X_new)):
    df_list.append(pd.DataFrame(X_new[i].reshape(1, -1)))

df1 = pd.concat(df_list, ignore_index=True)

C:\Users\10734646\AppData\Local\Temp\ipykernel_30948\3126000704.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df1 = pd.concat(df_list, ignore_index=True)


In [61]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.096694,0.247408,0.007467,0.070245,0.079638,-0.419919,0.110220,0.440731,-0.189659,-0.133010,...,0.327258,0.111083,-0.095554,0.059468,0.460939,0.229979,0.137533,-0.316116,0.153895,-0.056839
1,-0.081406,0.209151,0.003771,0.069471,0.066523,-0.363658,0.090415,0.375751,-0.165921,-0.111006,...,0.278293,0.097315,-0.086061,0.050415,0.398917,0.203075,0.128789,-0.272150,0.139326,-0.049364
2,-0.103763,0.271139,0.001458,0.080835,0.083567,-0.438707,0.123798,0.458259,-0.198397,-0.131708,...,0.337826,0.119017,-0.099257,0.066661,0.490509,0.250769,0.141994,-0.327448,0.173004,-0.060960
3,-0.141169,0.353473,0.009104,0.110795,0.107878,-0.609442,0.157689,0.642561,-0.284307,-0.181964,...,0.474423,0.160429,-0.140168,0.086719,0.679549,0.333807,0.209333,-0.458483,0.231156,-0.095240
4,-0.109976,0.278378,0.011767,0.078739,0.086394,-0.472701,0.122647,0.494857,-0.213510,-0.145033,...,0.364753,0.128002,-0.106139,0.064519,0.512535,0.255308,0.157394,-0.349711,0.173721,-0.065743


In [62]:
df1.isnull().sum()

0     68
1     68
2     68
3     68
4     68
      ..
95    68
96    68
97    68
98    68
99    68
Length: 100, dtype: int64

In [63]:
df1['Output']=y

In [64]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.096694,0.247408,0.007467,0.070245,0.079638,-0.419919,0.110220,0.440731,-0.189659,-0.133010,...,0.111083,-0.095554,0.059468,0.460939,0.229979,0.137533,-0.316116,0.153895,-0.056839,0
1,-0.081406,0.209151,0.003771,0.069471,0.066523,-0.363658,0.090415,0.375751,-0.165921,-0.111006,...,0.097315,-0.086061,0.050415,0.398917,0.203075,0.128789,-0.272150,0.139326,-0.049364,0
2,-0.103763,0.271139,0.001458,0.080835,0.083567,-0.438707,0.123798,0.458259,-0.198397,-0.131708,...,0.119017,-0.099257,0.066661,0.490509,0.250769,0.141994,-0.327448,0.173004,-0.060960,1
3,-0.141169,0.353473,0.009104,0.110795,0.107878,-0.609442,0.157689,0.642561,-0.284307,-0.181964,...,0.160429,-0.140168,0.086719,0.679549,0.333807,0.209333,-0.458483,0.231156,-0.095240,0
4,-0.109976,0.278378,0.011767,0.078739,0.086394,-0.472701,0.122647,0.494857,-0.213510,-0.145033,...,0.128002,-0.106139,0.064519,0.512535,0.255308,0.157394,-0.349711,0.173721,-0.065743,0


In [66]:
df1.dropna(inplace=True)

In [67]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.096694,0.247408,0.007467,0.070245,0.079638,-0.419919,0.110220,0.440731,-0.189659,-0.133010,...,0.111083,-0.095554,0.059468,0.460939,0.229979,0.137533,-0.316116,0.153895,-0.056839,0
1,-0.081406,0.209151,0.003771,0.069471,0.066523,-0.363658,0.090415,0.375751,-0.165921,-0.111006,...,0.097315,-0.086061,0.050415,0.398917,0.203075,0.128789,-0.272150,0.139326,-0.049364,0
2,-0.103763,0.271139,0.001458,0.080835,0.083567,-0.438707,0.123798,0.458259,-0.198397,-0.131708,...,0.119017,-0.099257,0.066661,0.490509,0.250769,0.141994,-0.327448,0.173004,-0.060960,1
3,-0.141169,0.353473,0.009104,0.110795,0.107878,-0.609442,0.157689,0.642561,-0.284307,-0.181964,...,0.160429,-0.140168,0.086719,0.679549,0.333807,0.209333,-0.458483,0.231156,-0.095240,0
4,-0.109976,0.278378,0.011767,0.078739,0.086394,-0.472701,0.122647,0.494857,-0.213510,-0.145033,...,0.128002,-0.106139,0.064519,0.512535,0.255308,0.157394,-0.349711,0.173721,-0.065743,0


In [68]:
df1.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [70]:
## Independent Feature
X=df1.drop('Output',axis=1)

In [113]:
X

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.096694,0.247408,0.007467,0.070245,0.079638,-0.419919,0.110220,0.440731,-0.189659,-0.133010,...,0.327258,0.111083,-0.095554,0.059468,0.460939,0.229979,0.137533,-0.316116,0.153895,-0.056839
1,-0.081406,0.209151,0.003771,0.069471,0.066523,-0.363658,0.090415,0.375751,-0.165921,-0.111006,...,0.278293,0.097315,-0.086061,0.050415,0.398917,0.203075,0.128789,-0.272150,0.139326,-0.049364
2,-0.103763,0.271139,0.001458,0.080835,0.083567,-0.438707,0.123798,0.458259,-0.198397,-0.131708,...,0.337826,0.119017,-0.099257,0.066661,0.490509,0.250769,0.141994,-0.327448,0.173004,-0.060960
3,-0.141169,0.353473,0.009104,0.110795,0.107878,-0.609442,0.157689,0.642561,-0.284307,-0.181964,...,0.474423,0.160429,-0.140168,0.086719,0.679549,0.333807,0.209333,-0.458483,0.231156,-0.095240
4,-0.109976,0.278378,0.011767,0.078739,0.086394,-0.472701,0.122647,0.494857,-0.213510,-0.145033,...,0.364753,0.128002,-0.106139,0.064519,0.512535,0.255308,0.157394,-0.349711,0.173721,-0.065743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,-0.121694,0.313412,0.010548,0.085954,0.093606,-0.496902,0.139874,0.520090,-0.226638,-0.151562,...,0.383636,0.132506,-0.106473,0.074800,0.564172,0.291837,0.157322,-0.373358,0.201518,-0.070691
5560,-0.116994,0.285874,0.018572,0.078273,0.080667,-0.497264,0.127030,0.521655,-0.228304,-0.156724,...,0.380452,0.123457,-0.111927,0.063754,0.539336,0.267751,0.170410,-0.375766,0.185304,-0.074183
5561,-0.051550,0.121204,-0.004131,0.033944,0.028157,-0.199394,0.046392,0.200702,-0.081720,-0.059165,...,0.151751,0.042689,-0.051670,0.017461,0.217906,0.109665,0.070018,-0.136156,0.074926,-0.030772
5562,-0.108695,0.281246,0.002549,0.078581,0.087194,-0.464482,0.128182,0.484825,-0.212494,-0.140536,...,0.356167,0.122150,-0.105529,0.066828,0.515615,0.260001,0.152440,-0.346147,0.174182,-0.065803


In [72]:
y=df1['Output']

In [114]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5559    1
5560    0
5561    0
5562    0
5563    0
Name: Output, Length: 5496, dtype: int64

In [103]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [104]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [105]:
classifier.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [106]:
y_pred=classifier.predict(X_test)

In [107]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.9681818181818181


In [108]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       943
           1       0.94      0.83      0.88       157

    accuracy                           0.97      1100
   macro avg       0.96      0.91      0.93      1100
weighted avg       0.97      0.97      0.97      1100



In [116]:
classifier = RandomForestClassifier()

In [117]:
classifier.fit(X,y)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [118]:
import pickle

In [123]:
with open('model.pkl','wb') as file:
    pickle.dump(classifier,file)
print("Model saved as model.pkl")

with open('model.pkl','rb') as file:
    load_model=pickle.load(file)

# Test the loaded model
print(
    load_model.predict([[2,9,6]])
    )

Model saved as model.pkl


ValueError: X has 3 features, but RandomForestClassifier is expecting 100 features as input.

In [115]:
!jupyter nbconvert --to script DL-2.ipynb

[NbConvertApp] Converting notebook DL-2.ipynb to script
[NbConvertApp] Writing 6850 bytes to DL-2.py


In [122]:
model.save("word2vec.model")

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

dataset = pd.read_csv('hiring.csv')

In [44]:
dataset

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,NaN,8.0,9,50000
1,NaN,8.0,6,45000
2,five,6.0,7,60000
3,two,10.0,10,65000
4,seven,9.0,6,70000
5,three,7.0,10,62000
6,ten,NaN,7,72000
7,eleven,7.0,8,80000


In [45]:
dataset.columns

Index(['experience', 'test_score(out of 10)', 'interview_score(out of 10)',
       'salary($)'],
      dtype='object')

In [46]:
dataset.isnull().sum()

experience                    2
test_score(out of 10)         1
interview_score(out of 10)    0
salary($)                     0
dtype: int64

In [47]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   experience                  6 non-null      object 
 1   test_score(out of 10)       7 non-null      float64
 2   interview_score(out of 10)  8 non-null      int64  
 3   salary($)                   8 non-null      int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 388.0+ bytes


In [48]:
median_value = dataset['test_score(out of 10)'].median()
dataset['test_score(out of 10)'].fillna(median_value, inplace=True)

C:\Users\10734646\AppData\Local\Temp\ipykernel_22876\811167548.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['test_score(out of 10)'].fillna(median_value, inplace=True)


In [49]:
dataset.isnull().sum()

experience                    2
test_score(out of 10)         0
interview_score(out of 10)    0
salary($)                     0
dtype: int64

In [50]:
#Converting words to integer values
def convert_to_int(word):
    word_dict = {'one':1, 'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'seven':7, 'eight':8,
                'nine':9, 'ten':10, 'eleven':11, 'twelve':12, 'zero':0, 0: 0}
    return word_dict[word]

In [51]:
word_to_num ={'one':1, 'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'seven':7, 'eight':8,
                'nine':9, 'ten':10, 'eleven':11, 'twelve':12, 'zero':0, 0: 0}

In [53]:
dataset['experience'] = dataset['experience'].map(word_to_num)

In [54]:
dataset

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,NaN,8.0,9,50000
1,NaN,8.0,6,45000
2,5.0,6.0,7,60000
3,2.0,10.0,10,65000
4,7.0,9.0,6,70000
5,3.0,7.0,10,62000
6,10.0,8.0,7,72000
7,11.0,7.0,8,80000


In [59]:
df=dataset.copy()

In [60]:
df

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,NaN,8.0,9,50000
1,NaN,8.0,6,45000
2,5.0,6.0,7,60000
3,2.0,10.0,10,65000
4,7.0,9.0,6,70000
5,3.0,7.0,10,62000
6,10.0,8.0,7,72000
7,11.0,7.0,8,80000


In [61]:
median_val = df['experience'].mode()
df['experience'].fillna(median_val, inplace=True)

C:\Users\10734646\AppData\Local\Temp\ipykernel_22876\1980687869.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['experience'].fillna(median_val, inplace=True)


In [62]:
df

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,2.0,8.0,9,50000
1,3.0,8.0,6,45000
2,5.0,6.0,7,60000
3,2.0,10.0,10,65000
4,7.0,9.0,6,70000
5,3.0,7.0,10,62000
6,10.0,8.0,7,72000
7,11.0,7.0,8,80000


In [63]:
median_val = dataset['experience'].mode()
dataset['experience'].fillna(median_val, inplace=True)

C:\Users\10734646\AppData\Local\Temp\ipykernel_22876\1760287025.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['experience'].fillna(median_val, inplace=True)


In [64]:
X=dataset.iloc[:,:3]

In [65]:
X

,experience,test_score(out of 10),interview_score(out of 10)
0,2.0,8.0,9
1,3.0,8.0,6
2,5.0,6.0,7
3,2.0,10.0,10
4,7.0,9.0,6
5,3.0,7.0,10
6,10.0,8.0,7
7,11.0,7.0,8


In [23]:
#Converting words to integer values
def convert_to_int(word):
    word_dict = {'one':1, 'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'seven':7, 'eight':8,
                'nine':9, 'ten':10, 'eleven':11, 'twelve':12, 'zero':0, 0: 0}
    return word_dict[word]
X['experience'] = X['experience'].apply(lambda x : convert_to_int(x))

In [68]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   experience                  8 non-null      float64
 1   test_score(out of 10)       8 non-null      float64
 2   interview_score(out of 10)  8 non-null      int64  
dtypes: float64(2), int64(1)
memory usage: 324.0 bytes


In [66]:
y=df['salary($)']

In [67]:
y

0    50000
1    45000
2    60000
3    65000
4    70000
5    62000
6    72000
7    80000
Name: salary($), dtype: int64

In [69]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [72]:
#Fitting model with trainig data
model = regressor.fit(X, y)


In [73]:
# Save the model
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved as model.pkl")


Model saved as model.pkl


In [76]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Test the loaded model
print(
    loaded_model.predict([[2,9,6]])
    )

[47693.86944432]


C:\Users\10734646\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [77]:
!pip freeze > requirements.txt

In [78]:
!pip install pipreqs

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/798.3 kB ? eta -:--:--
   ---- ----------------------------------- 92.2/798.3 kB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 143.4/798.3 kB 1.4 MB/s eta 0:00:01
   ---------- ----------------------------- 204.8/798.3 kB 1.6 MB/s eta 0:00:01
   ----------- ---------------------------- 225.3/798.3 kB 1.4 MB/s eta 0:00:01
   ------------- -------------------------- 276.5/798.3 kB 1.2 MB/s eta 0:00:01
   --------------- ------------------------ 307.2/798.3 kB 1.1 MB/s eta 0:00:01
   --------------- ---------------------- 317.4/798.3 kB 936.6 kB/s eta 0:00:01
   ---------------- --------------------- 337.9/798.3 kB 873.8 kB/s eta 0:00:01
   ----------------- -------------------- 358.4/798.3 kB 857.5 kB/s eta 0:00:01
   ----------------- -------------------- 368.6/798.3 kB 791.2 kB/s eta 0:00:01
   ------------------ ------

In [80]:
!pipreqs . --force

INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in .\requirements.txt


In [81]:
import pkg_resources
import sys

# Get all imported modules
imports = {mod.key for mod in pkg_resources.working_set}
with open("requirements.txt", "w") as f:
    for mod in imports:
        f.write(mod + "\n")

In [83]:
!jupyter nbconvert --to script model.ipynb


[NbConvertApp] Converting notebook model.ipynb to script
[NbConvertApp] Writing 1153 bytes to model.py
